In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from itertools import islice
import io
import json
from sklearn.utils import shuffle

2023-01-08 19:16:11.611845: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
yorumdf = pd.read_csv('./tabloYorumlar.csv')
yorum_datab = yorumdf['Baslik'].values.tolist()
yorum_datay = yorumdf['Yorum'].values.tolist()

In [4]:
yorum_metinleri = [str(s) for s in yorum_datab + yorum_datay]
len(yorum_metinleri)

508202

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

In [6]:
tokenizer.fit_on_texts(yorum_metinleri)

In [7]:
tokenizer_json = tokenizer.to_json()
with io.open('tf_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))


In [26]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return islice(iterable, n)

In [36]:
n_items_20000 = [s[0] for s in list(take(20000, tokenizer.word_index.items()))]
len(n_items_20000)

20000

In [40]:
wordDict = dict()
for i in range(len(n_items_20000)):    
    wordDict[n_items_20000[i]] = i + 1

In [42]:
with open('third_tokenizer.json', 'w', encoding='utf-8') as file:
    json.dump(wordDict, file, ensure_ascii=False)

In [62]:
yorumdf.head()

,Puan,Baslik,Yorum
0,four,sağlıklı ısınma,radyatörlü ısınmanın daha sağlıklı olduğunu dü...
1,five,süper,radyatörlü ısıtıcıları hiçbir şeye değişmem di...
2,five,klasik audio technica standart berrak ses,fiyatını hakkeden iyi bir kulaklık
3,four,şık ve teknolojik bir aksesuar,ürün başarılı ve güzel bir tamamlayıcı aksesua...
4,four,Şık ve kullanışlı,Ürün oldukça güzel tasarlanmış ve kullanışlı F...


In [67]:
yorumdf.loc[yorumdf['Puan'] == "five", 'Puan'] = 1.0
yorumdf.loc[yorumdf['Puan'] == "four", 'Puan'] = None
yorumdf.loc[yorumdf['Puan'] == "three", 'Puan'] = 0.0
yorumdf.loc[yorumdf['Puan'] == "two", 'Puan'] = 0.0
yorumdf.loc[yorumdf['Puan'] == "one", 'Puan'] = 0.0

yorumdf['Yorum'] = yorumdf['Baslik'] + ' ' + yorumdf['Yorum']
del yorumdf['Baslik']
yorumdf.loc[yorumdf['Yorum'].str.len() < 10, 'Yorum'] = None

yorumdf = yorumdf.dropna()
yorumdf

,Puan,Yorum
1,1.0,süper radyatörlü ısıtıcıları hiçbir şeye değiş...
2,1.0,klasik audio technica standart berrak ses fiya...
5,0.0,idare eder idare eder işte ben mikrofonlu diye...
7,0.0,Berbat bir urun Cok sesli Basligi donmuor cok ...
8,0.0,alma olumsuz yorumların hepsine katılıyorum ta...
...,...,...
254089,0.0,UFO LİNE 1200W GEREKSİZ Evimizde küçük odamızd...
254090,1.0,kargo inanılmaz hızlı Ürünü cuma günü sipariş ...
254091,0.0,İşimizi görüyor Ürünü alalı 1 ay kadar oldu ya...
254094,1.0,harika bir ürün hiç düşünmeden aldım 10 10 200...


In [68]:
yorum_pos = yorumdf.loc[yorumdf['Puan'] == 1.0]
yorum_pos

,Puan,Yorum
1,1.0,süper radyatörlü ısıtıcıları hiçbir şeye değiş...
2,1.0,klasik audio technica standart berrak ses fiya...
15,1.0,Şaşkınım Siparişim hızlıca elime ulaştı Hemen ...
17,1.0,harika ilginize cok teşekkür ederim makina gay...
22,1.0,teşekkürler mağaza çok ilgili herşey on numara...
...,...,...
254086,1.0,ısıtması iyi ürünü buradan aldım elektirik sar...
254088,1.0,zaman Ürünü gerçekten çooook arştırarak burada...
254090,1.0,kargo inanılmaz hızlı Ürünü cuma günü sipariş ...
254094,1.0,harika bir ürün hiç düşünmeden aldım 10 10 200...


In [69]:
yorum_neg = yorumdf.loc[yorumdf['Puan'] == 0.0]
yorum_neg

,Puan,Yorum
5,0.0,idare eder idare eder işte ben mikrofonlu diye...
7,0.0,Berbat bir urun Cok sesli Basligi donmuor cok ...
8,0.0,alma olumsuz yorumların hepsine katılıyorum ta...
10,0.0,Arzu Bende kullanıyorum güzel bir ürün memnun ...
12,0.0,Çok zayıf Hevesle aldım yaklaşık 1 5 kilo port...
...,...,...
254058,0.0,3 puan tasarım olarak güzel ekran kalitesi tat...
254061,0.0,Parasına göre iş yapar Parasına göre gayet alı...
254064,0.0,beklentimin çok üzerinde 4 gündür kullanıyoruz...
254089,0.0,UFO LİNE 1200W GEREKSİZ Evimizde küçük odamızd...


In [74]:
yorumdf = pd.concat([yorum_pos.head(39776), yorum_neg])
yorumdf = shuffle(yorumdf)
yorumdf

,Puan,Yorum
23080,1.0,Çak iyi fiyata çok iyi ürün Tam zamanında kusu...
52724,1.0,Mükemmel Ürün siparişi verdim ve 24 saatten kı...
247365,0.0,beğenmedim ses seviyesi beklediğimden düşük bu...
230593,0.0,Klavye Bildiğiniz klavye klavye olduğu için öv...
25222,0.0,Ucuz çok iyi değil ama idare eder TV nin plast...
...,...,...
214867,0.0,Kaliteli Urun Hizli Kargo Pazar gunu siparis v...
67074,1.0,Harika bir ürün Umduğumdan çok daha iyi görünt...
39316,1.0,sağlam ve güzel paketleme sağlam ve güzel pake...
157457,0.0,Kargo rezil Mağaza ilgili değil


In [ ]:
# save to csv file
yorumdf.to_csv('yorumlar.csv', index=False)